# [roberta-classical-chinese-base-ud-goeswith](https://huggingface.co/KoichiYasuoka/roberta-classical-chinese-base-ud-goeswith)による句間リンク抽出(チューニング中)

In [ ]:
!pip install transformers ufal.chu_liu_edmonds deplacy
text1,text2="切切偲偲","怡怡如也"
bert="KoichiYasuoka/roberta-classical-chinese-base-ud-goeswith"
def dsp(h,p,t,l):
  import deplacy
  u="# text = "+"".join(t)+"\n"
  for i,j in enumerate(t,1):
    q=l[p[i,h[i]]].split("|")
    u+="\t".join([str(i),j,"_",q[0],"_","|".join(q[1:-1]),str(h[i]),q[-1],"_","SpaceAfter=No"])+"\n"
  deplacy.serve(u+"\n",port=None)
from transformers import AutoTokenizer,AutoModelForTokenClassification
tokenizer=AutoTokenizer.from_pretrained(bert)
model=AutoModelForTokenClassification.from_pretrained(bert)
import numpy,torch,ufal.chu_liu_edmonds
w=tokenizer([text1,text2],add_special_tokens=False,return_offsets_mapping=True)
v=[tokenizer.cls_token_id]+sum(w["input_ids"],[])+[tokenizer.sep_token_id]
x=[v[0:i]+[tokenizer.mask_token_id]+v[i+1:]+[j] for i,j in enumerate(v[1:-1],1)]
with torch.no_grad():
  e=model(input_ids=torch.tensor(x)).logits.numpy()[:,1:-2,:]
l=model.config.id2label
r=[1 if i==0 else -1 if j.endswith("|root") else 0 for i,j in sorted(l.items())]
e+=numpy.where(numpy.add.outer(numpy.identity(e.shape[0]),r)==0,0,numpy.nan)
g=model.config.label2id["X|_|goeswith"]
r=numpy.tri(e.shape[0])
for i in range(e.shape[0]):
  for j in range(i+2,e.shape[1]):
    r[i,j]=r[i,j-1] if numpy.nanargmax(e[i,j-1])==g else 1
e[:,:,g]+=numpy.where(r==0,0,numpy.nan)
m=numpy.full((e.shape[0]+1,e.shape[1]+1),numpy.nan)
m[1:,1:]=numpy.nanmax(e,axis=2).transpose()
p=numpy.zeros(m.shape)
p[1:,1:]=numpy.nanargmax(e,axis=2).transpose()
for i in range(1,m.shape[0]):
  m[i,0],m[i,i],p[i,0]=m[i,i],numpy.nan,p[i,i]
h=ufal.chu_liu_edmonds.chu_liu_edmonds(m)[0]
t=sum([[y[s:e] for s,e in x if s<e] for x,y in zip(w["offset_mapping"],[text1,text2])],[])
dsp(h,p,t,l)
mm=m.copy()
n=len(w[0])+1
mm[1:n,n:]=mm[n:,1:n]=numpy.nan
h=ufal.chu_liu_edmonds.chu_liu_edmonds(mm)[0]
dsp(h,p,t,l)
w=[i for i,j in enumerate(h) if j==0]
mm[w]=m[w]
mm[:,0]+=numpy.where(mm[:,0]==numpy.nanmax(mm[w,0]),0,numpy.nan)
mm[w]+=[0 if i==0 or j==0 else numpy.nan for i,j in enumerate(h)]
h=ufal.chu_liu_edmonds.chu_liu_edmonds(mm)[0]
dsp(h,p,t,l)